In [ ]:
import json
from typing import Dict, List, Literal
from datetime import datetime
import openai  

In [ ]:

MOCK_CLAIMS = {
    "denied": {
        "policy_number": "POL-123456",
        "date_of_loss": "2025-10-15",
        "claim_status": "DENIED",
        "exclusion_clause": "Section B, Item 4 - Water damage from flood",
        "adjuster_name": "John Smith",
        "claim_number": "CLM-789012",
        "estimated_damage": 25000.00,
        "details": "Property damage due to flooding during heavy rainfall"
    },
    "reduced": {
        "policy_number": "POL-234567",
        "date_of_loss": "2025-10-20",
        "claim_status": "REDUCED",
        "total_damage": 15000.00,
        "deductible": 5000.00,
        "depreciation": 2000.00,
        "final_payout": 8000.00,
        "claim_number": "CLM-890123",
        "adjuster_name": "Sarah Johnson",
        "details": "Roof damage due to hailstorm"
    },
    "approved": {
        "policy_number": "POL-345678",
        "date_of_loss": "2025-10-25",
        "claim_status": "APPROVED",
        "total_payout": 2500.00,
        "repair_shop": "AutoFix Pro Services",
        "payment_date": "2025-11-05",
        "claim_number": "CLM-901234",
        "adjuster_name": "Mike Wilson",
        "details": "Vehicle rear bumper damage from minor collision"
    }
}

In [ ]:
class ClaimsExplanationBot:
    def __init__(self):
        self.TONE_PROMPTS = {
            "standard": "Write in a professional, balanced tone that maintains objectivity while being clear and respectful.",
            "empathetic": "Write with high empathy and understanding, acknowledging the customer's situation while maintaining professionalism. Use supportive and reassuring language.",
            "brief": "Write concisely and directly, focusing on the key facts and next steps. Use short sentences and bullet points where appropriate."
        }
    
    def extract_claim_info(self, claim_data: Dict) -> Dict:
        """
        Step 1: Extract and structure key information from the claim report
        """
        return {
            "decision": claim_data["claim_status"],
            "claim_number": claim_data["claim_number"],
            "final_amount": claim_data.get("final_payout") or claim_data.get("total_payout", 0),
            "primary_reason": self._determine_primary_reason(claim_data),
            "next_steps": self._determine_next_steps(claim_data)
        }
    
    def _determine_primary_reason(self, claim_data: Dict) -> str:
        """Helper method to determine the primary reason for the claim decision"""
        if claim_data["claim_status"] == "DENIED":
            return f"Policy Exclusion: {claim_data['exclusion_clause']}"
        elif claim_data["claim_status"] == "REDUCED":
            return f"Adjustments: ${claim_data['depreciation']} depreciation, ${claim_data['deductible']} deductible"
        else:
            return "Claim meets policy coverage requirements"
    
    def _determine_next_steps(self, claim_data: Dict) -> str:
        """Helper method to determine the next steps based on claim status"""
        if claim_data["claim_status"] == "APPROVED":
            return f"Contact {claim_data['repair_shop']} to schedule your repairs"
        else:
            return "Call our Appeals Hotline at 1-800-APPEALS if you believe this decision should be reviewed"
    
    def generate_explanation(self, claim_data: Dict, tone: Literal["standard", "empathetic", "brief"] = "standard") -> str:
        """
        Step 2 & 3: Generate a customer-friendly explanation with the specified tone
        """
      
        info = self.extract_claim_info(claim_data)
        
       
        base_prompt = f"""
        Claim Number: {info['claim_number']}
        Decision: {info['decision']}
        Amount: ${info['final_amount']:,.2f}
        Reason: {info['primary_reason']}
        
        {self.TONE_PROMPTS[tone]}
        
        Generate a clear explanation for the customer about their claim decision.
        """
        
       
        return self._generate_mock_response(info, tone)
    
    def _generate_mock_response(self, info: Dict, tone: str) -> str:
        """Generate a mock response for demonstration purposes"""
        responses = {
            "DENIED": {
                "standard": f"Regarding claim {info['claim_number']}: After careful review, we must deny this claim based on {info['primary_reason']}. {info['next_steps']}.",
                "empathetic": f"I understand how disappointing this news may be. After thoroughly reviewing claim {info['claim_number']}, we found that {info['primary_reason']}. While we can't approve the claim, we want to help you understand your options. {info['next_steps']}.",
                "brief": f"Claim {info['claim_number']} - DENIED\n• Reason: {info['primary_reason']}\n• Next step: {info['next_steps']}"
            },
            "REDUCED": {
                "standard": f"Your claim {info['claim_number']} has been processed with a final payment of ${info['final_amount']:,.2f}. {info['primary_reason']}. {info['next_steps']}.",
                "empathetic": f"We've carefully reviewed your claim {info['claim_number']} and worked to maximize your coverage. While some adjustments were necessary ({info['primary_reason']}), we've approved a payment of ${info['final_amount']:,.2f}. {info['next_steps']}.",
                "brief": f"Claim {info['claim_number']} - APPROVED (Adjusted)\n• Payment: ${info['final_amount']:,.2f}\n• Adjustments: {info['primary_reason']}\n• Next step: {info['next_steps']}"
            },
            "APPROVED": {
                "standard": f"Good news! Your claim {info['claim_number']} has been approved for ${info['final_amount']:,.2f}. {info['next_steps']}.",
                "empathetic": f"We're pleased to inform you that your claim {info['claim_number']} has been fully approved. You'll receive ${info['final_amount']:,.2f}, and we're here to help you get back to normal. {info['next_steps']}.",
                "brief": f"Claim {info['claim_number']} - APPROVED\n• Payment: ${info['final_amount']:,.2f}\n• Next step: {info['next_steps']}"
            }
        }
        
        return responses[info['decision']][tone]

In [ ]:

bot = ClaimsExplanationBot()


def demonstrate_claim_scenario(claim_type: str):
    print(f"=== Demonstrating {claim_type.upper()} Claim Scenario ===\n")
    claim_data = MOCK_CLAIMS[claim_type]
    
    print("Original Claim Data:")
    print(json.dumps(claim_data, indent=2))
    print("\nExplanations in Different Tones:\n")
    
    for tone in ["standard", "empathetic", "brief"]:
        print(f"--- {tone.title()} Tone ---")
        explanation = bot.generate_explanation(claim_data, tone)
        print(explanation)
        print()

In [ ]:

for scenario in ["denied", "reduced", "approved"]:
    demonstrate_claim_scenario(scenario)

=== Demonstrating DENIED Claim Scenario ===

Original Claim Data:
{
  "policy_number": "POL-123456",
  "date_of_loss": "2025-10-15",
  "claim_status": "DENIED",
  "exclusion_clause": "Section B, Item 4 - Water damage from flood",
  "adjuster_name": "John Smith",
  "claim_number": "CLM-789012",
  "estimated_damage": 25000.0,
  "details": "Property damage due to flooding during heavy rainfall"
}

Explanations in Different Tones:

--- Standard Tone ---
Regarding claim CLM-789012: After careful review, we must deny this claim based on Policy Exclusion: Section B, Item 4 - Water damage from flood. Call our Appeals Hotline at 1-800-APPEALS if you believe this decision should be reviewed.

--- Empathetic Tone ---
I understand how disappointing this news may be. After thoroughly reviewing claim CLM-789012, we found that Policy Exclusion: Section B, Item 4 - Water damage from flood. While we can't approve the claim, we want to help you understand your options. Call our Appeals Hotline at 1-800-

In [ ]:
def get_user_claim_input():
    """Get claim details from user input"""
    print("Please enter the claim details:")
    
    claim_data = {}
    
    # Basic claim information
    claim_data["claim_number"] = input("Claim Number (e.g., CLM-123456): ")
    claim_data["policy_number"] = input("Policy Number (e.g., POL-123456): ")
    claim_data["date_of_loss"] = input("Date of Loss (YYYY-MM-DD): ")
    
    # Claim status
    print("\nSelect claim status:")
    print("1. APPROVED")
    print("2. DENIED")
    print("3. REDUCED")
    status_choice = input("Enter choice (1-3): ")
    status_map = {"1": "APPROVED", "2": "DENIED", "3": "REDUCED"}
    claim_data["claim_status"] = status_map.get(status_choice, "DENIED")
    
    
    if claim_data["claim_status"] == "APPROVED":
        claim_data["total_payout"] = float(input("\nTotal Payout Amount ($): "))
        claim_data["repair_shop"] = input("Repair Shop Name: ")
        claim_data["payment_date"] = input("Payment Date (YYYY-MM-DD): ")
    
    elif claim_data["claim_status"] == "DENIED":
        claim_data["exclusion_clause"] = input("\nExclusion Clause (e.g., Section B, Item 4 - Water damage): ")
        claim_data["estimated_damage"] = float(input("Estimated Damage Amount ($): "))
    
    elif claim_data["claim_status"] == "REDUCED":
        claim_data["total_damage"] = float(input("\nTotal Damage Amount ($): "))
        claim_data["deductible"] = float(input("Deductible Amount ($): "))
        claim_data["depreciation"] = float(input("Depreciation Amount ($): "))
        claim_data["final_payout"] = claim_data["total_damage"] - claim_data["deductible"] - claim_data["depreciation"]
    
    claim_data["adjuster_name"] = input("\nAdjuster Name: ")
    claim_data["details"] = input("Claim Details (brief description): ")
    
    return claim_data

In [ ]:
def interactive_claims_bot():
    """Run the interactive claims bot"""
    print("Welcome to the Claims Explanation Bot!")
    print("=====================================")
    
    
    claim_data = get_user_claim_input()
    
    print("\nProcessing your claim...\n")
    
    
    bot = ClaimsExplanationBot()
    
    
    print("Explanations in Different Tones:")
    print("================================")
    
    for tone in ["standard", "empathetic", "brief"]:
        print(f"\n--- {tone.title()} Tone ---")
        explanation = bot.generate_explanation(claim_data, tone)
        print(explanation)
    
    return claim_data  

In [ ]:

if __name__ == "__main__":
    user_claim = interactive_claims_bot()

Welcome to the Claims Explanation Bot!
Please enter the claim details:

Select claim status:
1. APPROVED
2. DENIED
3. REDUCED

Processing your claim...

Explanations in Different Tones:

--- Standard Tone ---
Good news! Your claim clm - 12876 has been approved for $200,000.00. Contact shi to schedule your repairs.

--- Empathetic Tone ---
We're pleased to inform you that your claim clm - 12876 has been fully approved. You'll receive $200,000.00, and we're here to help you get back to normal. Contact shi to schedule your repairs.

--- Brief Tone ---
Claim clm - 12876 - APPROVED
• Payment: $200,000.00
• Next step: Contact shi to schedule your repairs
